In [93]:
import numpy as np
import pandas as pd
import sys
import os
import copy
import plotly.express as px
import importlib
import pickle
os.chdir('/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment')
import rmsd_functions as rmsd
importlib.reload(rmsd)

<module 'rmsd_functions' from '/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/rmsd_functions.py'>

In [95]:
pdb_dir = '/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/ORs_uncentered'
pdb_files = os.listdir(pdb_dir)
pdb_files = [pdb for pdb in pdb_files if ".pdb" in pdb]
data = {}
for file in pdb_files:
    name = file.split('.')[0]
    # for i in name.split('_'): # if there are multiple Olfr names for a single file, take the first 'Olfr' string
    #     if 'Olfr' in name: 
    #         name = name
    #         break
    name = name.split('-')[0]
    atom, coord, resid = rmsd.get_coordinates_pdb(f'{pdb_dir}/{file}')
    coord -= rmsd.centroid(coord)
    data[name] = {}
    data[name]["atom"] = atom
    data[name]["coord"] = coord
    data[name]["resid"] = resid


In [96]:
backbone = copy.deepcopy(data)
backbone = rmsd.get_backbone(backbone)
print(backbone)


Reordered Olfr73_mOR atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr876 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr983 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr653 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Reordered Olfr168 atoms : ['C' 'CA' 'CB' 'CE' 'CG' 'N' 'O' 'SD' 'C' 'CA']
Backbone atoms extracted for  Olfr73_mOR
Backbone atoms extracted for  Olfr876
Backbone atoms extracted for  Olfr983
Backbone atoms extracted for  Olfr653
Backbone atoms extracted for  Olfr168
{'Olfr73_mOR': {'atom': array(['C', 'CA', 'N', ..., 'CA', 'N', 'O'], dtype='<U3'), 'coord': array([[ 19.15944417,   6.48073569, -41.45233618],
       [ 18.69244417,   7.83473569, -41.96733618],
       [ 19.89344417,   8.58273569, -42.38833618],
       ...,
       [ 12.66144417, -25.14026431,  25.79866382],
       [ 11.19744417, -25.09726431,  25.94966382],
       [ 14.30044417, -26.17726431,  27.13266382]]), 'resid': array([  1,  

In [97]:
backbone = rmsd.trim_resid(backbone, start=25, end=285)

Resid trimed from 25 to 285 for Olfr73_mOR
Resid trimed from 25 to 285 for Olfr876
Resid trimed from 25 to 285 for Olfr983
Resid trimed from 25 to 285 for Olfr653
Resid trimed from 25 to 285 for Olfr168


In [98]:
small_Olfr = []
for Olfr in backbone:
    if backbone[Olfr]['resid'].max() != 285:
        small_Olfr.append(Olfr)
for Olfr in small_Olfr: 
    data.pop(Olfr)
    backbone.pop(Olfr)

print(backbone)


{'Olfr73_mOR': {'atom': array(['C', 'CA', 'N', ..., 'CA', 'N', 'O'], dtype='<U3'), 'coord': array([[24.12044417,  5.25073569,  2.97866382],
       [24.19644417,  6.31673569,  1.88666382],
       [24.43244417,  7.67473569,  2.41666382],
       ...,
       [ 5.77044417, -0.66726431,  8.81266382],
       [ 5.14944417, -0.50126431,  7.50266382],
       [ 4.74444417, -0.92426431, 10.98066382]]), 'resid': array([ 25,  25,  25, ..., 285, 285, 285])}, 'Olfr876': {'atom': array(['C', 'CA', 'N', ..., 'CA', 'N', 'O'], dtype='<U3'), 'coord': array([[15.23806522, 17.24989007,  2.93406235],
       [16.20506522, 18.40489007,  3.22106235],
       [17.49206522, 18.26989007,  2.53706235],
       ...,
       [-1.56093478,  3.64989007,  8.97806235],
       [-0.79593478,  4.82989007,  8.56206235],
       [-2.40493478,  3.28989007, 11.19006235]]), 'resid': array([ 25,  25,  25, ..., 285, 285, 285])}, 'Olfr983': {'atom': array(['C', 'CA', 'N', ..., 'CA', 'N', 'O'], dtype='<U3'), 'coord': array([[  0.43916904

In [105]:
# aligns the sequences in the input data by kabsch method
# Backbone is used to align, but rotation is asigned onto the full lengthed data 
data = rmsd.align_data(backbone, align="Olfr73_mOR", rotate_data=data)

        


Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr876
Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr983
Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr653
Rotation calculated for Olfr73_mOR and applied to ROTATE DATA  Olfr168
RETURN new rotated data


In [94]:
# Center Olfr73 on its binding site
# Known binding site residues come from https://www.nature.com/articles/s42003-019-0384-8

f = '/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb/Olfr73_mOR-EG.pdb'
coord = rmsd.get_coordinates_pdb(f)

atomlist = [102, 105, 106, 109, 181, 182, 199, 203, 208, 259, 260, 273, 277, 280]
xatom = []
yatom = []
zatom = []
for i in range(len(coord[0])):

    if coord[2][i] in atomlist:
        xatom.append(coord[1][i][0])
        yatom.append(coord[1][i][1])
geo_center = [np.mean(np.asarray(xatom)), 
        zatom.append(coord[1][i][2])
              np.mean(np.asarray(yatom)), 
              np.mean(np.asarray(zatom))]

print(geo_center)



[2.7519206349206353, 6.195484126984126, -6.388984126984128]


In [ ]:
# Adjust center of each aligned protein by center of binding pocket on Olfr73
for k in data.keys():
    data[k]['coord'] -= geo_center

In [106]:
data_df = rmsd.dict_to_pdDataFrame(data)

In [107]:
# Save the aligned OR xyz coordinates for all AF pdb 
data_df.to_csv('/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb_df_test.csv')

In [108]:
data_df = pd.read_csv('/home/users/jvs15/project-protein-fold/dual_cnn/OR_alignment/AF_pdb_df_test.csv', index_col=0)

In [109]:
rmsd.df_plot_all(data_df[data_df.models.isin(data_df['models'].unique()[:5])])

In [104]:
pickle_file = open('existing_symb_to_unip.p', 'rb')
d = pickle.load(pickle_file)
print(d)



{'OR10A2': 'Q9H208', 'OR10A3': 'P58181', 'OR10A5': 'Q9H207', 'OR10A6': 'Q8NH74', 'OR10A7': 'Q8NGE5', 'OR10AG1': 'Q8NH19', 'OR10C1': 'Q96KK4', 'OR10G3': 'Q8NGC4', 'OR10G4': 'Q8NGN3', 'OR10G6': 'Q8NH81', 'OR10G7': 'Q8NGN6', 'OR10G8': 'Q8NGN5', 'OR10H1': 'Q9Y4A9', 'OR10H3': 'O60404', 'OR10H4': 'Q8NGA5', 'OR10J1': 'P30954', 'OR10J3': 'Q5JRS4', 'OR10J5': 'Q8NHC4', 'OR10K1': 'Q8NGX5', 'OR10K2': 'Q6IF99', 'OR10P1': 'Q8NGE3', 'OR10Q1': 'Q8NGQ4', 'OR10R2': 'Q8NGX6', 'OR10T2': 'Q8NGX3', 'OR10X1': 'Q8NGY0', 'OR10Z1': 'Q8NGY1', 'OR11A1': 'Q9GZK7', 'OR11G2': 'Q8NGC1', 'OR11H1': 'Q8NG94', 'OR11H12': 'B2RN74', 'OR11H2': 'Q8NH07', 'OR11H4': 'Q8NGC9', 'OR11H6': 'Q8NGC7', 'OR11H7': 'Q8NGC8', 'OR12D2': 'P58182', 'OR12D3': 'Q9UGF7', 'OR13A1': 'Q8NGR1', 'OR13C2': 'Q8NGS9', 'OR13C3': 'Q8NGS6', 'OR13C4': 'Q8NGS5', 'OR13C5': 'Q8NGS8', 'OR13C8': 'Q8NGS7', 'OR13C9': 'Q8NGT0', 'OR13D1': 'Q8NGV5', 'OR13H1': 'Q8NG92', 'OR14A2': 'Q96R54', 'OR14J1': 'Q9UGF5', 'OR14K1': 'Q8NGZ2', 'OR1A1': 'Q9P1Q5', 'OR1A2': 'Q9Y585',